In [1]:
import tensorflow as tf

简单的两个数组相加案例

In [2]:
a = tf.constant([1,2], name='a')
b = tf.constant([3,4], name='b')
result = a + b
sess = tf.Session()
sess.run(result)  

array([4, 6])

# 1 计算图Graph
---
```python
import tensorflow as tf
tf.Graph()
```

## 1.1 计算图的使用
---
每个计算图相互独立

In [5]:
# 创建两个计算图，并定义一个初始化变量v
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable('v', shape=[1], initializer=tf.zeros_initializer)
    
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('v', shape=[1], initializer=tf.ones_initializer)
    
# 获取g1中的v
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse=True):
        print('Graph1 v = ', sess.run(tf.get_variable('v')))

# 获取g2中的v
with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse=True):
        print('Graph2 v = ', sess.run(tf.get_variable('v')))

Graph1 v =  [0.]
Graph2 v =  [1.]


指定图在GPU计算

In [9]:
g = tf.Graph()
a = tf.constant([1,2], name='a')
b = tf.constant([3,4], name='b')

with g.device('/gpu:0'):
    result = a + b
    with tf.Session() as sess:
        print(sess.run(result))

[4 6]


## 1.2 Tensor
---
### 1.2.1 introduction
---
1. tensor
    - 0阶张量为标量(scalar)，对应一个数
    - 一阶张量为向量(vector)，对应一维数组
    - n阶张量对应n维数组
2. tensor的数据结构
    - name: `name:src_output`
    - shape:
    - dtype:
3. tensor支持的数据类型
    - tf.float32, tf.float64
    - tf.int8, tf.int16, tf.int32, tf.int64, tf.uint8
    - tf.bool
    - tf.complex64, tf.complex128
4. 张量的使用
    - 对中间计算结果的引用
    - 可以通过name获取保存的计算结果

In [31]:
a = tf.constant([1,2], name='a')
b = tf.constant([3,4], name='b')
result = tf.add(a, b, name='add_ab')

In [34]:
result
# '节点名:该节点的第几个输出'，如果重复赋值，以前的不会被覆盖，会出现`add_ab_1:0`

<tf.Tensor 'add_ab:0' shape=(2,) dtype=int32>

In [57]:
a

<tf.Tensor 'a_12:0' shape=(2,) dtype=int32>

## 1.3 Session
---
tensorflow 通过使用Session来执行定义好的运算。
```python
sess = tf.Session()
# 执行定义好的张量
sess.run()

sess.close()
```
or
```python
with tf.Session() as sess:
    sess.run()
```

### 1.3.1 定义默认会话
---


**方法1**：使用as_default创建一个context manager，在内部可以直接使用tensor_name.eval()执行。

In [83]:
# 使用as_default创建一个context manager，在内部可以直接使用tensor_name.eval()执行。
sess = tf.Session()
with sess.as_default():
    print(result.eval())

[4 6]


**方法2**：使用`tf.InteractiveSession()`函数,直接创建一个默认session，不用再去指定。

In [86]:
sess = tf.InteractiveSession()
print(result.eval())

sess.close()

[4 6]


**方法3**：使用`ConfigProto()`创建一个config对象，在session创建的时候指定config属性

In [89]:
config = tf.ConfigProto(allow_soft_placement=True,
                       log_device_placement=True)

sess1 = tf.InteractiveSession(config=config)
# sess2 = tf.Session(config=config)

## 1.4 实现神经网络
---


In [100]:
x = tf.constant([0.7, 0.9], shape=(2, 1))

In [101]:
w1 = tf.constant([[0.2, 0.3],[0.1,-0.5],[0.4,0.2]], shape=(3, 2))

In [102]:
w2 = tf.constant([0.6, 0.1, -0.2], shape=(1, 3))

In [103]:
a = tf.matmul(w1, x)
y = tf.matmul(w2, a)

In [104]:
y.eval()

array([[0.11600002]], dtype=float32)

### 1.4.1 variable
---
在tensorflow中，一个变量值在被使用之前，这个变量的**初始化过程**要被明确调用。变量定义类似于python中的函数定义，想要使用，需要调用。通过session调用。
```python
# 创建variable,并将变量加入到GraphKeys.VARIABLES,如果参数trainable=True,默认为True,则变量会被加入到GraphKeys.TRAINABLE_VARIABLES
v = tf.Variable()
# 初始化
v.initializer

# 获取已定义的变量列表
tf.global_variables()

# 获取可训练的变量列表
tf.trainable_variables

```

>  随机生成函数
---
```python
tf.random_normal
tf.truncated_normal
tf.random_uniform
tf.random.gamma
```

> 常数生成函数
---
```python
tf.zeros
tf.ones
tf.fill
tf.constant
```
### 1.4.2 前向传播
---
1. 定义输入值
2. 定义隐藏层1的权重
3. 定义隐藏层2的权重
4. 计算

In [14]:
import tensorflow as tf

# 定义输入值
x = tf.constant([0.7,0.9], shape=(2, 1), dtype=tf.float32)

# 随机生成权重值，采用正太分布
w1 = tf.Variable(tf.random_normal((3, 2), stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal((1, 3), stddev=1, seed=1))

# 定义计算过程
h = tf.matmul(w1, x)
y = tf.matmul(w2, h)

# 定义全局变量初始化
init_op = tf.global_variables_initializer()

# 开始计算
with tf.Session() as sess:
#     sess.run(w1.initializer)
#     sess.run(w2.initializer)
    sess.run(init_op)
    print(sess.run(y))

[[-3.7798672]]


### 1.4.3 placeholder
---
用placeholder管理常量
```python
# 创建
v = tf.placeholder(dtype, shape=None, name=None)
```

In [3]:
import tensorflow as tf

# 定义输入值
x = tf.placeholder(tf.float32, shape=(2, 1), name='input')

# 随机生成权重值，采用正太分布
w1 = tf.Variable(tf.random_normal((3, 2), stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal((1, 3), stddev=1, seed=1))

# 定义计算过程
h = tf.matmul(w1, x)
y = tf.matmul(w2, h)

# 定义全局变量初始化
init_op = tf.global_variables_initializer()

# 开始计算
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(y, feed_dict={x: [[0.7],[0.9]]}))

[[-3.7798672]]


定义多维输入值,这里定义三个样本

In [8]:
import tensorflow as tf
import numpy as np

# 定义输入值（三个样本）
x = tf.placeholder(tf.float32, shape=(2, 3), name='input')

# 随机生成权重值，采用正太分布
w1 = tf.Variable(tf.random_normal((3, 2), stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal((1, 3), stddev=1, seed=1))

# 定义计算过程
h = tf.matmul(w1, x)
y = tf.matmul(w2, h)

# 定义全局变量初始化
init_op = tf.global_variables_initializer()

# 定义输入值
input_data = np.array([[0.7, 0.9],
                      [0.1, 0.4],
                      [0.5, 0.8]]).T
# 开始计算
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(y, feed_dict={x: input_data}))

[[-3.7798674 -1.8407464 -3.4529805]]


### 1.4.4 反向传播
---
这里采用交叉熵
$$H(p,q)=-\sum_{i=1}^np(x_i)log(q(x_i))$$

### 1.4.5 完整的代码
---
<b style="color:red">认为书中此部分有bug,待补充</b>

In [49]:
import tensorflow as tf
import numpy as np

# 定义batch的大小
batch_size = 8

# 定义输入值,这里不具体指定样本的数量
x = tf.placeholder(tf.float32, shape=(2, None), name='input')
# 定义输出的label
y_ = tf.placeholder(tf.float32, shape=(1, None), name='target')

# 随机生成权重值，采用正太分布
w1 = tf.Variable(tf.random_normal((3, 2), stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal((1, 3), stddev=1, seed=1))

# 定义计算过程
h = tf.matmul(w1, x)
y = tf.matmul(w2, h)

y = tf.sigmoid(y)


# 定义全局变量初始化
init_op = tf.global_variables_initializer()

# 定义输入值
input_data = np.array([[0.7, 0.9],
                      [0.1, 0.4],
                      [0.5, 0.8]]).T
# 开始计算
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(y, feed_dict={x: input_data}))


[[0.02231633 0.13696304 0.0306801 ]]


In [45]:
import math


In [46]:
tf.sigmoid?

In [48]:
1/ (1 + np.e**(1.84))

0.13705129257545964

In [52]:
print(1e-10)

1e-10
